In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/데청캠 3조 공유 드라이브/Model/성민/data/sentencePairData.csv')
df

,Unnamed: 0,first_sentence,second_sentence,label
0,0,"둘째, ICT 산학협력 SW 교에서는 오답 노트 시스템을 개발했습니다.",네 좋아해요,0
1,1,금융업계에서의 올해 최고 이슈는 ‘핀테크’입니다.,"은행을 비롯해 모든 금융업계는 IT인프라를 늘리고, 관련 부서를 신설하는 등 핀테크...",1
2,2,이러한 열정과 도전정신은 저의 인생 모토가 되었고 어떠한 위기와 역경에도 일단 부딪...,학생회를 통해 리더십과 책임감을 발휘할 수 있었으며 혼자가 아닌 팀으로 하여 더 효...,0
3,3,업무일지 작성으로 사내 인력과 상황에의 대처능력과 조직적응력을 빠르게 나타내겠습니다.,"선수들이 거세게 항의를 하면 판정을 번복하기도 하고, 스로인 반칙을 보고도 그냥 넘...",0
4,4,카드는 여행 구성에 상관없이 반드시 쓰는 혜택을 구성하고 O2O 서비스를 통해 고객...,신한 판클럽은 야 놀자 하고 제휴는 맺었지만 여행지나 렌터카 등 부가적인 서비스는 ...,1
...,...,...,...,...
325657,325657,아직 배울 게 많은 신입사원입니다.,친구 결혼식도 갈 겸 회사 인턴도 할 겸 맞추려고 하거든요,0
325658,325658,제가 소극적이였을 때는 할 수 있는 것도 못하고 피해를 보는 경우가 있었지만 지금은...,주변 상가는 장사가 잘 되나요?,0
325659,325659,더불어 같은 조원이였던 중국학우를 통해 다른 나라 문화의 도시계획에 대해 깊은 인상...,네 데워드려요,0
325660,325660,이론적인 내용에만 만족하는 것이 아닌 직접 제작하며 배운 회로는 이론과 너무나 달랐...,시설관리업무는 처음이라 ‘할 수 있는 것’이라는 확신도 없습니다.,0


In [ ]:
data_set = list()
for i in range(0, len(df)):
    data_set.append([df['first_sentence'][i],df['second_sentence'][i], df['label'][i]])

In [ ]:
test_per = 0.2
train_size = int(len(data_set)*(1-test_per))
print(train_size) 
train_data = data_set[:train_size]
test_data = data_set[train_size: len(data_set)]

260529


In [ ]:
import torch
torch.cuda.get_device_name(0)

'Tesla P100-PCIE-16GB'

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

     |████████████████████████████████| 46.9 MB 69 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 4.3 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595735 sha256=c5aa8712190d11f55759662763964b760eb0bb6824f6cf27444c980f0efe7892
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 4.1 MB/s 
     |████████████████████████████████| 2.6 MB 4.2 MB/s 
     |████████████████████████████████| 3.3 MB 55.4 MB/s 
     |████████████████████████████████| 895 kB 41.5 MB/s 
     |████████████████████████████████| 636 kB 64.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstal

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-700w41kc
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-700w41kc
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12771 sha256=46cab82b7782a76f788f1fc16d6df01b80a5cbe6b253b21a6f1f53a26b734294
  Stored in directory: /tmp/pip-ephem-wheel-cache-4uogan16/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [ ]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
##GPU 사용 시
device = torch.device("cuda:0")  # "cuda:0"

In [ ]:
print(device)

cuda:0


In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [ ]:
class myBertDataset(Dataset):
    def __init__(self, dataset, bert_tokenizer, max_len, pad, pair):
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length = max_len, pad=pad, pair= pair)
        self.sentences = [transform(dataset[i][0:2]) for i in range(0,len(dataset))]
        self.labels = [np.int32(dataset[i][2]) for i in range(0,len(dataset))]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
    
    def __len__(self): 
        return (len(self.labels))

In [ ]:
## Setting parameters
max_len = 512
batch_size = 8
warmup_ratio = 0.1
num_epochs = 4
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
data_Train = myBertDataset(train_data, tok, max_len, True, True)
data_Test = myBertDataset(test_data, tok, max_len, True, True)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_Train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_Test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768, 
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1   
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))[0]
        if self.dr_rate:
            pooler = pooler[:,0,:] 
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel, dr_rate=0.2).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
OUTPUT_PATH = '/content/gdrive/MyDrive/데청캠 3조 공유 드라이브/Model/성민/output/'

In [ ]:
class EarlyStopping:
    def __init__(self, patience=2, verbose=False, delta=0, path=OUTPUT_PATH+'checkpoint_final.pt'):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [ ]:
test_losses = []
avg_test_losses = []
early_stopping = EarlyStopping(patience = 2, verbose = True)
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        #print(out.shape)
        #print(label.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        test_losses.append(loss.item())
        test_acc += calc_accuracy(out, label)
    test_loss = np.average(test_losses)
    test_losses = []
    early_stopping(test_loss, model)

    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

    if early_stopping.early_stop:
        print('Early stopping')
        break
    torch.save(model.state_dict(), OUTPUT_PATH+'checkpoint_epoch'+str(e)+'.pt')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/32567 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 0.6626768112182617 train acc 0.625
epoch 1 batch id 201 loss 0.6407122611999512 train acc 0.746268656716418
epoch 1 batch id 401 loss 0.4520088732242584 train acc 0.8095386533665836
epoch 1 batch id 601 loss 0.1840997189283371 train acc 0.831946755407654
epoch 1 batch id 801 loss 0.439775675535202 train acc 0.848314606741573
epoch 1 batch id 1001 loss 0.8665885329246521 train acc 0.8577672327672328
epoch 1 batch id 1201 loss 0.06059405952692032 train acc 0.8648001665278934
epoch 1 batch id 1401 loss 0.04298820346593857 train acc 0.8694682369735903
epoch 1 batch id 1601 loss 0.4648098051548004 train acc 0.8735165521549032
epoch 1 batch id 1801 loss 0.49559348821640015 train acc 0.8786091060521932
epoch 1 batch id 2001 loss 0.02482970617711544 train acc 0.8821214392803598
epoch 1 batch id 2201 loss 0.014247480779886246 train acc 0.8834620626987733
epoch 1 batch id 2401 loss 0.06834141165018082 train acc 0.8861932528113287
epoch 1 batch id 2601 loss 0.0064069908112

In [ ]:
model = model.load_state_dict(torch.load('checkpoint.pt'))